## This notebook inputs the missing values using Moving Average

In [5]:
import pandas as pd
from datetime import datetime
import numpy as np

#### Choose wether the imputation should be done on the data set with daily or weekly granularity

In [6]:
INPUT, OUTPUT = 'CompleteIndexes.csv','MovingAverageImputed.csv'
#INPUT, OUTPUT = 'CompleteWeeklyIndexes.csv', 'MovingAverageWeeklyImputed.csv'

In [7]:
df = pd.read_csv(INPUT, parse_dates=True, index_col=0)
df = df.loc['2000-01-01':'2015-01-01'] #Selecting training data
df.head(30)

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n255_Close,n255_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.05,40.634806,NaN,NaN,NaN
2000-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.62,NaN,NaN,NaN,NaN
2000-01-03,1455.219971,9.318000e+08,24.209999,0.0,11357.509766,169750000.0,3790.550049,1.510070e+09,NaN,NaN,...,17369.630859,0.0,996.770020,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-01-04,1399.420044,1.009000e+09,27.010000,0.0,10997.929688,178420000.0,3546.199951,1.511840e+09,19002.859375,0.0,...,17072.820313,0.0,955.969971,0.0,NaN,0.99,NaN,NaN,NaN,NaN
2000-01-05,1402.109985,1.085500e+09,26.410000,0.0,11122.650391,203190000.0,3507.310059,1.735670e+09,18542.550781,0.0,...,15846.719727,0.0,930.260010,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-01-06,1403.449951,1.092300e+09,25.730000,0.0,11253.259766,176550000.0,3340.810059,1.598320e+09,18168.269531,0.0,...,15153.230469,0.0,922.460022,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-01-07,1441.469971,1.225200e+09,21.719999,0.0,11522.559570,184900000.0,3529.600098,1.634930e+09,18193.410156,0.0,...,15405.629883,0.0,943.880005,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.78,NaN,NaN,NaN,NaN
2000-01-10,1457.599976,1.064800e+09,21.709999,0.0,11572.200195,168180000.0,3717.409912,1.691710e+09,NaN,NaN,...,15848.150391,0.0,960.219971,0.0,NaN,0.00,NaN,NaN,NaN,NaN


In [8]:
def imput_moving_average_to_col(df_to_input, num_values):
    for index, row in df_to_input.iterrows():
        if pd.isna(row[0]):
            #print("_______________", row)
            count_prev = 0
            mean = 0
            index_prev = df_to_input.index.get_loc(index)
            while count_prev < num_values:
                index_prev -= 1
                if index_prev < 0:
                    break
                if not pd.isna(df_to_input.iloc[index_prev][0]):
                    count_prev += 1
                    #print(df_to_input.iloc[index_prev])
                    mean += df_to_input.iloc[index_prev]
            count_next = 0
            index_next = df_to_input.index.get_loc(index)
            while count_next < num_values:
                index_next += 1
                if index_next >= len(df_to_input):
                    break
                if not pd.isna(df_to_input.iloc[index_next][0]):
                    count_next += 1
                    #print(df_to_input.iloc[index_next])
                    mean += df_to_input.iloc[index_next]
            mean = mean / (count_prev+count_next)
            #print("MEAN" , mean, "----------------")
            df_to_input.at[index, df_to_input.columns[0]]  = mean
    return df_to_input

In [9]:
for col in df.columns:
    df_imp = imput_moving_average_to_col(df[[col]],3)
    df[col] = df_imp
    print("Column", col, "was imputed.")
df.head()

Column sp500_Close was imputed.
Column sp500_Volume was imputed.
Column vix_Close was imputed.
Column vix_Volume was imputed.
Column dji_Close was imputed.
Column dji_Volume was imputed.
Column ndx_Close was imputed.
Column ndx_Volume was imputed.
Column n255_Close was imputed.
Column n255_Volume was imputed.
Column ftse_Close was imputed.
Column ftse_Volume was imputed.
Column hsi_Close was imputed.
Column hsi_Volume was imputed.
Column n100_Close was imputed.
Column n100_Volume was imputed.
Column Overall EMV Tracker was imputed.
Column infectious_daily_infect_emv_index was imputed.
Column GPR was imputed.
Column trade_US Trade Policy Uncertainty was imputed.
Column trade_Japanese Trade Policy Uncertainty was imputed.
Column trade_Trade Policy EMV Fraction was imputed.


,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n255_Close,n255_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2000-01-01,1418.916667,1.008767e+09,25.876666,0.0,11159.363282,1.837867e+08,3614.686686,1.585860e+09,18571.226562,0.0,...,16763.056966,0.0,961.000000,0.0,23.477003,1.05,40.634806,70.148805,69.294997,0.029394
2000-01-02,1418.916667,1.008767e+09,25.876666,0.0,11159.363282,1.837867e+08,3614.686686,1.585860e+09,18571.226562,0.0,...,16763.056966,0.0,961.000000,0.0,23.477003,2.62,41.223446,70.148805,69.294997,0.029394
2000-01-03,1455.219971,9.318000e+08,24.209999,0.0,11357.509766,1.697500e+08,3790.550049,1.510070e+09,18571.226562,0.0,...,17369.630859,0.0,996.770020,0.0,23.477003,0.00,41.223446,70.148805,69.294997,0.029394
2000-01-04,1399.420044,1.009000e+09,27.010000,0.0,10997.929688,1.784200e+08,3546.199951,1.511840e+09,19002.859375,0.0,...,17072.820313,0.0,955.969971,0.0,23.477003,0.99,41.223446,70.148805,69.294997,0.029394
2000-01-05,1402.109985,1.085500e+09,26.410000,0.0,11122.650391,2.031900e+08,3507.310059,1.735670e+09,18542.550781,0.0,...,15846.719727,0.0,930.260010,0.0,23.477003,0.00,41.321553,70.148805,69.294997,0.029394


In [11]:
df.to_csv(OUTPUT, index = True)